<center>
<h2> </h2>
<h2>ALTeGraD 2024<br>
<h2>Lab Session 3: Large Language Models</h2>
<h5>October 22, 2024</h5>
<h4><b>Emilio PICARD</b> </h4>
<br>
</center>
[Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B)
<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 29, 2024 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).

<hr style="border:5px solid gray"> </hr>

<hr style="border:5px solid gray"> </hr>

* Please submit your file to Moodle or [here](https://docs.google.com/forms/d/e/1FAIpQLSfDFjQcvjKxLn42Kxpw5ek2Ce_lHzMCVSST8R2AJcQct6Np2A/viewform)

<br><br>
In this lab, we will:

* fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

* To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

* Compare the results before and after instructin tuning.

* Fintune the model again on perference dataset using [DPO](https://huggingface.co/docs/trl/main/dpo_trainer#dpo-trainer)(direct perference optimization)
 <br>


# <b>Part 1 Finetuning Qwen2.5-0.5B using HuggingFace's Transfromers</b>
In this section, we will fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

## <b>Preparing the environment and installing libraries:<b>

In [1]:
!nvidia-smi

Tue Oct 29 21:44:52 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 561.09         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   40C    P8              2W /   56W |     806MiB /   6141MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

In [3]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## <b>Loading the model and the tokenizer:<b>

In this section, we will load the QWEN model while using the BitsAndBytes library for quantization.

In [4]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
# MODEL_NAME = "unsloth/Llama-3.2-1B" # Try Llama if you want

bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [62]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel() # fill the gap: get the number of trainable parameters: trainable_params
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [63]:
print(print_trainable_parameters(model))

trainable params: 1081344 || all params: 316200832 || trainable%: 0.34198012483408013
None


## <b>Configuring LoRA:<b>

In [64]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    # target_modules=["query_key_value"],
    bias="none",
    task_type="CAUSAL_LM" # fill the gaph
)

model = get_peft_model(model,config) # fill the gap, using lora weights
print_trainable_parameters(model)

trainable params: 1081344 || all params: 316200832 || trainable%: 0.34198012483408013


## <b>Test the model before finetuning:<b>

In [65]:
prompt = "<human>: What equipment do I need for rock climbing?  \n <assistant>: , with an empty response from the assistant"   # # fill the gap, prompt of the format: "<human>: What equipment do I need for rock climbing?  \n <assistant>: ", with an empty response from the assistant
print(prompt)


generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

<human>: What equipment do I need for rock climbing?  
 <assistant>: , with an empty response from the assistant


**Question**: what does the temperature do in the above cell?

**Answer**: The `temperature` parameter controls the randomness of the predictions made by the model. A high temperature (>1) increases randomness by making the probability distribution of the next word more uniform. A low temperature (<1) decreases randomness by making the probability distribution sharper.

In [66]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: What equipment do I need for rock climbing?  
 <assistant>: , with an empty response from the assistant: , I need a pair of climbing shoes, a pair of climbing gloves, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of cl

## <b>Loading the question/answer dataset from HuggingFace:<b>

In [68]:
data = load_dataset("HuggingFaceH4/helpful-instructions")
pd.DataFrame(data["train"])

,instruction,demonstration,meta
0,"Hi, I want to learn to play horseshoes. Can yo...","I can, but maybe I should begin by telling you...",{'source': 'helpful-anthropic-raw'}
1,How do I teach kids to meditate?,Great question! That’s a really useful skill t...,{'source': 'helpful-anthropic-raw'}
2,Can you tell me the steps for getting a harbor...,Sure. I believe you’ll need a copy of the mari...,{'source': 'helpful-anthropic-raw'}
3,How can I store food if I don't have a pantry?,"You could store the food in a refrigerator, th...",{'source': 'helpful-anthropic-raw'}
4,what are some good novels for a 9 year old?,"That depends on the 9 year old, but if they li...",{'source': 'helpful-anthropic-raw'}
...,...,...,...
147701,"Given the following sentence, classify it into...",Fact,{'source': 'helpful-self-instruct-raw'}
147702,A person wants to write a book. he/she writes ...,Chapter 1 - The History of China\nChapter 2 - ...,{'source': 'helpful-self-instruct-raw'}
147703,Tell me how you would make a popular app game.,I would make a game that is similar to 2048. T...,{'source': 'helpful-self-instruct-raw'}
147704,Describe your dream house to me.\n\nOutput:,My dream house is a two-story building with a ...,{'source': 'helpful-self-instruct-raw'}


## <b>Preparing the finetuning data:<b>

In [69]:
data["train"][0]["instruction"]

'Hi, I want to learn to play horseshoes. Can you teach me?'

In [70]:
def generate_prompt(data_point):
    return f"<human>: {data_point['instruction']} \n <assistant>: {data_point['demonstration']}"
    
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)


Map:   0%|          | 0/147706 [00:00<?, ? examples/s]

## <b>Finetuning:<b>

In [71]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,   # try more steps if you can
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,3.288000
2,2.105500
3,2.793700
4,2.484200
5,2.855900
6,2.603300
7,3.081800
8,2.188800
9,2.901900
10,2.596800


TrainOutput(global_step=200, training_loss=2.0192769241333006, metrics={'train_runtime': 701.6291, 'train_samples_per_second': 1.14, 'train_steps_per_second': 0.285, 'total_flos': 156500683637760.0, 'train_loss': 2.0192769241333006, 'epoch': 0.005416164543078819})

In [72]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs --port 6008

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6008 (pid 18668), started 2:35:56 ago. (Use '!kill 18668' to kill it.)

## <b>Test the model after the finetuning:<b>

In [73]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: What equipment do I need for rock climbing?  
 <assistant>: , with an empty response from the assistant, the user may be asking for a specific type of equipment, such as a harness, a rope, or a climbing harness.  If the user is asking for a specific type of equipment, the assistant may also provide a list of equipment that is required for that type of equipment.  For example, if the user is asking for a harness, the assistant may provide a list of equipment that is required for a harness, such as a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness, a harness
CPU times: total: 17.6 s
Wall time: 18 s


In [74]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question} \n <assistant>: "  # Transforme la question en un prompt avec une réponse vide
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()


In [75]:
prompt = "What program can I use to edit video clips I took with my phone?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))



- What program can I use to edit video clips I took with my phone? 

1. Open the video file in your video editing software. 
2. Use the video editing software’s tools to edit the video. 
3. Save the edited video as a new file. 
4. Use the video editing software’s tools to edit the video again. 
5. Save the edited video as a new file. 
6. Use the video editing software’s tools to edit the video again. 
7. Save the edited video as a new file. 
8. Use the video editing software’s tools to edit the video again. 
9. Save the edited video as a new file. 
10. Use the video editing software’s tools to edit the video again. 
11. Save the edited video as a new file. 
12. Use the video editing software’s tools to edit the video again. 
13. Save the edited video as a new file. 
14. Use the video editing software’s tools to edit the video again.



- Do you know the reasons as to why people love coffee so much? 

Coffee is a popular drink in many countries, and it is often associated with a sense o

# Part 2: DPO
In this part we will use the instrcution tuned LLM to do direct preference optimization. see the paper: https://arxiv.org/abs/2305.18290

DPO involves tuning the model on preference data, normally consists of a prompt, a prefered answer and a rejected answer.

The core advantage of DPO is its ability to simultaneously bypass the explicit reward modeling step while avoiding the complexities of reinforcement learning optimization.

## Test the model before DPO:


In [76]:
prompt_2 = "<system> You are a helpful assistant\n<human>: How a house is made ?\n<assistant>: "
print(prompt_2)

<system> You are a helpful assistant
<human>: How a house is made ?
<assistant>: 


In [77]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<system> You are a helpful assistant
<human>: How a house is made ?
<assistant>: 1. Build a foundation
2. Install windows and doors
3. Install electrical and plumbing
4. Install heating and cooling systems
5. Install appliances
6. Install security systems
7. Install security systems
8. Install security systems
9. Install security systems
10. Install security systems
11. Install security systems
12. Install security systems
13. Install security systems
14. Install security systems
15. Install security systems
16. Install security systems
17. Install security systems
18. Install security systems
19. Install security systems
20. Install security systems
21. Install security systems
22. Install security systems
23. Install security systems
24. Install security systems
25. Install security systems
26. Install security systems
27. Install security systems
28. Install security systems
29. Install security systems
30.
CPU times: total: 18.5 s
Wall time: 19.3 s


## Loading the preference data from Huggingface:

In [78]:
data_dpo = load_dataset("CultriX/llama70B-dpo-dataset")
pd.DataFrame(data_dpo["train"])

,system,question,chosen,rejected
0,You are a helpful assistant,Can you taste this dish and tell me if it need...,I'm not capable of physically tasting the dish...,I can taste the dish remotely and I think it n...
1,You are a helpful assistant,Can you smell this perfume and tell me if it's...,"As a digital AI assistant, I don't have a phys...","I can smell the perfume, and it smells great. ..."
2,You are a helpful assistant,Please turn off the lights in my room.,"I'm a large language model, I don't have the c...",I can turn off the lights in your room remotel...
3,You are a helpful assistant,What is the primary reason why many approximat...,The primary reason why many approximate attent...,Approximate attention methods haven't gained w...
4,You are a helpful assistant,What are the properties of melanin that contri...,Melanin's photoprotective role can be attribut...,Melanin's photoprotective role is due to its a...
...,...,...,...,...
2174,You are a helpful assistant,What is the sound barrier?,"The sound barrier, also known as the sonic bar...",The sound barrier is a physical wall that prev...
2175,You are a helpful assistant,Is spinach a great source of dietary iron?,"Spinach is a good source of dietary iron, but ...","Spinach is a poor source of dietary iron, and ..."
2176,You are a helpful assistant,What challenges arise in training large langua...,Training large language models (LLMs) poses se...,The main challenge in training LLMs is the lac...
2177,You are a helpful assistant,Does the theory of evolution explain the origi...,The theory of evolution explains how life on E...,The theory of evolution fully explains the ori...


## Preparing the data:

Similar to instruction tuning, we should first construct our prompt, which should follow the DPO format, see: https://huggingface.co/docs/trl/main/dataset_formats#preference

In [79]:
def preprocess_data_dpo(data_point):
    return {"system": data_point["system"],
    "question": data_point["question"],
    "prompt": f"<system> {data_point['system']} <human>: {data_point['question']}  \n <assistant>: ",
    "chosen": data_point["chosen"],
    "rejected": data_point["rejected"],
    } # fill the gap, using dpo format

data_dpo = data_dpo['train'].shuffle(seed=42).map(preprocess_data_dpo)

In [80]:
print(data_dpo)

Dataset({
    features: ['system', 'question', 'chosen', 'rejected', 'prompt'],
    num_rows: 2179
})


## Finetuning

**Question**: what is beta in dpo_args?

**Answer**: In `dpo_args`, beta is a parameter that typically controls the trade-off between different objectives or regularization terms in the optimization process. A high value of beta increases differences between the answers, whereas a low value make the difference more smooth.

In [1]:
!pip install transformers==4.45.2


  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.45.2-py3-none-any.whl (9.9 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.0
    Uninstalling transformers-4.46.0:
      Successfully uninstalled transformers-4.46.0


In [81]:
OUTPUT_DIR = "experiments_dpo"

training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
    beta=0.1
)

dpo_args = {
    "beta": 0.1,
}

trainer = DPOTrainer(
    model=model,
    train_dataset=data_dpo,
    args=training_args,
    tokenizer=tokenizer,
    beta=dpo_args["beta"],
) # fill the gap

model.config.use_cache = False
trainer.train()

C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\trl\trainer\dpo_trainer.py:660: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\trl\trainer\dpo_trainer.py:673: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\trl\trainer\dpo_trainer.py:708: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/2179 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,1.125700
2,0.695200
3,1.142100
4,0.925200
5,1.194300
6,0.928700
7,1.084300
8,0.859300
9,0.956100
10,0.706500


TrainOutput(global_step=200, training_loss=0.18405938989885726, metrics={'train_runtime': 2122.2159, 'train_samples_per_second': 0.377, 'train_steps_per_second': 0.094, 'total_flos': 0.0, 'train_loss': 0.18405938989885726, 'epoch': 0.36714089031665903})

## Test the model after DPO:

In [82]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<system> You are a helpful assistant
<human>: How a house is made ?
<assistant>:  A house is a structure consisting of a building envelope (e.g., walls, roof, foundation), interior spaces (e.g., living areas, bedrooms, bathrooms), and exterior elements (e.g., exterior walls, roof, foundation), designed to provide a comfortable, safe, and energy-efficient living and working environment. The process of constructing a house involves several stages, including planning, design, construction, and maintenance. Here’s a more detailed breakdown:

1. **Planning**: This phase involves identifying the purpose of the house (e.g., residential, commercial), location, and environmental considerations. It also involves determining the size, layout, and functional requirements of the space.

2. **Design**: This phase involves conceptualizing the design of the house, including the layout, materials, and visual elements. It also involves considering the energy efficiency, sustainability, and aesthetic asp

In [83]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question}\n<assistant>: "
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [84]:
prompt = "Do people dream in color or black and white?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Explain the concept of economic policies in simple terms"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

print('\n\n\n-', prompt, '\n')
prompt = "Explain the effects of globalization on the environment."
print(generate_response(prompt))

- Do people dream in color or black and white? 

Many people dream in color or black and white, depending on their individual experiences and visual perception. However, some individuals may also dream in other visual languages, such as grayscale or grayscale-based representations of color. This can be influenced by various factors, including cultural background, personal experiences, and cognitive processing.



- Explain the concept of economic policies in simple terms 

Economic policies are strategic decisions made by governments or other actors to influence the allocation of resources, income, and wealth within an economy. These policies can include initiatives such as fiscal policy (e.g., taxation, spending), monetary policy (e.g., interest rates, money supply), and regulatory policies (e.g., regulation of industries, antitrust enforcement). They aim to achieve economic stability, growth, efficiency, and equity by guiding the economy towards a balanced and sustainable development

Is the response improved after DPO?

**Answer**: After running the inference test, the responses provided by the DPO model are significantly more precise. There is a notable improvement in prediction quality, with responses showing a finer contextual understanding and enhanced relevance to the questions posed.

In [85]:
prompt = " What equipment do I need for rock climbing? "
print('-', prompt,'\n')
print(generate_response(prompt))

-  What equipment do I need for rock climbing?  

For rock climbing, I would recommend considering the following equipment: climbing harness, helmet, gloves, crampons (climbing shoes), ropes or anchor points, anchor points, safety gear such as helmets, gloves, and crampons, and other specialized gear such as belay devices and ropes. Additionally, it’s important to consult with a certified instructor or climbing expert to ensure proper safety and technique.
